This notebook is intended to make a faster version of this notebook: (**63x**)
* https://www.kaggle.com/byfone/h-m-trending-products-weekly/notebook 

The formula above is already well explained in this notebook  : 
* https://www.kaggle.com/lichtlab/0-0226-byfone-chris-combination-approach/notebook 

Please upvote those notebook notebook, and this notebook too if you find this helpful :D 

This notebook has reduce **63x** the processing time from **2 hours to 2 minutes**

In [ ]:
import numpy as np
import pandas as pd 
from datetime import datetime, timedelta
import gc

import cudf

### Read the transactions data

Changes: 
* read_csv using cudf
* article_id types from string to int32 
* reduce memory on customer_id 

In [ ]:
N = 12

In [ ]:
df  = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv',
                            usecols= ['t_dat', 'customer_id', 'article_id'], 
                            dtype={'article_id': 'int32', 't_dat': 'string', 'customer_id': 'string'})
df ['customer_id'] = df ['customer_id'].str[-16:].str.hex_to_int().astype('int64')

df['t_dat'] = cudf.to_datetime(df['t_dat'])
last_ts = df['t_dat'].max()

### Add the last day of billing week

Note: 
* In the original code, `ldbw` is the next nearest (or current) Tuesday from `t_dat` (since 2020-09-22 is Tuesday)
    
Changes: 
* Use `pandas.Series.dt.dayofweek` and `pd.TimedeltaIndex` 
* Do not use `last_ts`

Steps: 
1. Get the day of week, Monday = 0, Tuesday = 1 and so on 
2. Truncated t_dat into the Tuesday of t_dat week, saved as ldbw 
3. for t_dat Wednesday until Sunday add 7 days to get next Tuesday, do nothing for t_dat Monday and Tuesday

Time reduction: 
* Before: 55 min 47 sec
* After: 8 sec

In [ ]:
%%time
tmp = df[['t_dat']].copy().to_pandas()
tmp['dow'] = tmp['t_dat'].dt.dayofweek
tmp['ldbw'] = tmp['t_dat'] - pd.TimedeltaIndex(tmp['dow'] - 1, unit='D')
tmp.loc[tmp['dow'] >=2 , 'ldbw'] = tmp.loc[tmp['dow'] >=2 , 'ldbw'] + pd.TimedeltaIndex(np.ones(len(tmp.loc[tmp['dow'] >=2])) * 7, unit='D')

df['ldbw'] = tmp['ldbw'].values

### Count the number of transactions per week 

Changes: 
* Do reset index after groupby 
* df.join to df.merge 

In [ ]:
weekly_sales = df.drop('customer_id', axis=1).groupby(['ldbw', 'article_id']).count().reset_index()
weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})

In [ ]:
df = df.merge(weekly_sales, on=['ldbw', 'article_id'], how = 'left')

### Let's assume that in the target week sales will be similar to the last week of the training data

In [ ]:
weekly_sales = weekly_sales.reset_index().set_index('article_id')

df = df.merge(
    weekly_sales.loc[weekly_sales['ldbw']==last_ts, ['count']],
    on='article_id', suffixes=("", "_targ"))

df['count_targ'].fillna(0, inplace=True)
del weekly_sales

### Calculate sales rate adjusted for changes in product popularity 

In [ ]:
df['quotient'] = df['count_targ'] / df['count']

### Take supposedly popular products

In [ ]:
target_sales = df.drop('customer_id', axis=1).groupby('article_id')['quotient'].sum()
general_pred = target_sales.nlargest(N).index.to_pandas().tolist()
general_pred = ['0' + str(article_id) for article_id in general_pred]
general_pred_str =  ' '.join(general_pred)
del target_sales

In [ ]:
general_pred # Exactly same with the original notebook! 

In [ ]:
general_pred_str

### Fill in purchase dictionary

Changes: 
* avoid looping, use pandas series instead 
* Use dataframe instead of dictionary (faster on submission part)

Time reduction:
* Before: 39 min 09 sec
* After: 28 sec


In [ ]:
%%time
purchase_dict = {}

tmp = df.copy().to_pandas()
tmp['x'] = ((last_ts - tmp['t_dat']) / np.timedelta64(1, 'D')).astype(int)
tmp['dummy_1'] = 1 
tmp['x'] = tmp[["x", "dummy_1"]].max(axis=1)

a, b, c, d = 2.5e4, 1.5e5, 2e-1, 1e3
tmp['y'] = a / np.sqrt(tmp['x']) + b * np.exp(-c*tmp['x']) - d

tmp['dummy_0'] = 0 
tmp['y'] = tmp[["y", "dummy_0"]].max(axis=1)
tmp['value'] = tmp['quotient'] * tmp['y'] 

tmp = tmp.groupby(['customer_id', 'article_id']).agg({'value': 'sum'})
tmp = tmp.reset_index()

tmp = tmp.loc[tmp['value'] > 100]
tmp['rank'] = tmp.groupby("customer_id")["value"].rank("dense", ascending=False)
tmp = tmp.loc[tmp['rank'] <= 12]

# for customer_id in tmp['customer_id'].unique():
#     purchase_dict[customer_id] = {} 

# for customer_id, article_id, value in zip(tmp['customer_id'], tmp['article_id'], tmp['value']):
#     purchase_dict[customer_id][article_id] = value

purchase_df = tmp.sort_values(['customer_id', 'value'], ascending = False).reset_index(drop = True)
purchase_df['prediction'] = '0' + purchase_df['article_id'].astype(str) + ' '
purchase_df = purchase_df.groupby('customer_id').agg({'prediction': sum}).reset_index()
purchase_df['prediction'] = purchase_df['prediction'].str.strip()
purchase_df = cudf.DataFrame(purchase_df)

### Make a submission

Changes: 
* read_csv using cudf 
* avoid looping, use pandas series instead 
* Use dataframe instead of dictionary 
* Concatenate prediction string (tricks found in https://www.kaggle.com/cdeotte/recommend-items-purchased-together-0-021) 

Time reduction:
* Before: 15 min 44 sec
* After: 15 sec


In [ ]:
%%time
sub  = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv',
                            usecols= ['customer_id'], 
                            dtype={'customer_id': 'string'})

sub['customer_id2'] = sub['customer_id'].str[-16:].str.hex_to_int().astype('int64')

sub = sub.merge(purchase_df, left_on = 'customer_id2', right_on = 'customer_id', how = 'left',
               suffixes = ('', '_ignored'))

sub = sub.to_pandas()
sub['prediction'] = sub['prediction'].fillna(general_pred_str)
sub['prediction'] = sub['prediction'] + ' ' +  general_pred_str
sub['prediction'] = sub['prediction'].str.strip()
sub['prediction'] = sub['prediction'].str[:131]
sub = sub[['customer_id', 'prediction']]
sub.to_csv(f'submission.csv',index=False)
